ADD DOCUMENTATION

In [39]:
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime
import seaborn as sns

top_n_coins_input = 2 # Number of top coins to fetch, based on market cap, you can change this value

In [40]:
top_n_coins = min(50, max(1, top_n_coins_input)) # Limit to 50 coins
url = "https://api.coingecko.com/api/v3/coins/markets" # CoinGecko public API, no API key required but rate-limited so higher amount of requests will take longer
params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": top_n_coins,
    "page": 1,
}

response = requests.get(url, params=params)
top_coins = response.json()

print(f"Top {top_n_coins} cryptocurrencies by market cap:")
for i, coin in enumerate(top_coins):
    print(f"{i + 1}: {coin["name"]} ({coin["symbol"].upper()})")
    print(f"   Market Cap: ${coin["market_cap"]:,}")
    print(f"   Current Price: ${coin["current_price"]:,}")

Top 2 cryptocurrencies by market cap:
1: Bitcoin (BTC)
   Market Cap: $1,736,051,256,441
   Current Price: $87,415
2: Ethereum (ETH)
   Market Cap: $197,766,035,517
   Current Price: $1,636.95


In [ ]:

if top_n_coins > 20:
    print("Note: Loading more than 20 coins may take a while due to API limits.")

list_for_dataframe = []
logos_directory = "logos"
os.makedirs(logos_directory, exist_ok=True)

for coin in top_coins:
    retries = 0
    coin_id = coin["id"]
    
    # Fetch logo
    logo_data = requests.get(coin["image"]).content
    logo_path = os.path.join(logos_directory, f"{coin_id}.png")
    with open(logo_path, "wb") as f:
        f.write(logo_data)
        
    params = {
        "vs_currency": "usd",
        "days": "30",
        "interval": "daily",
    }
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    
    while retries < 3:
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if "prices" not in data:
                raise ValueError(f"No 'prices' in response: {data}")

            seen_dates = set()

            for day in data["prices"]:
                timestamp, price = day
                date_str = datetime.fromtimestamp(timestamp / 1000).strftime("%Y-%m-%d")
                if date_str not in seen_dates:
                    list_for_dataframe.append({
                        "name": coin["name"],
                        "date": date_str,
                        "price": price,
                    })
                    seen_dates.add(date_str)
            break
        except Exception as e:
            retries += 1
            print(f"Error fetching data for {coin["name"]}: {e}")
            if retries < 3:
                print(f"Retrying in 30 seconds... (attempt {retries}/3)")
                time.sleep(30)
            else:
                print(f"❌ Failed to fetch data for {coin["name"]} after 3 attempts.")
    
df = pd.DataFrame(list_for_dataframe)

sns.set_theme(style="whitegrid")

grouped = df.groupby("name")["price"]
mean_prices = grouped.mean()
std_prices = grouped.std()

# Calculate coefficient of variation (CV)
cv = std_prices / mean_prices

low_volatility_names = cv[cv < 0.01].index
df = df[~df["name"].isin(low_volatility_names)]
for name in low_volatility_names:
    print(f"Low volatility coin removed: {name}")
    
unique_names = df["name"].unique()
fig, axes = plt.subplots(len(unique_names), 1, figsize=(10, 4 * len(unique_names)), sharex=True)

if len(unique_names) == 1:
    axes = [axes]  # make it iterable if only 1 axis

for ax, name in zip(axes, unique_names):
    group = df[df["name"] == name].copy()
    group["pct_change"] = group["price"].pct_change() * 100

    ax.plot(group["date"], group["price"], marker='o', label=name)

    for x, y, pct in zip(group["date"], group["price"], group["pct_change"]):
        price_label = f"${y:,.2f}"
        ax.text(x, y, price_label, fontsize=8, ha='center', va='bottom', rotation=45)

        if not pd.isna(pct):
            pct_label = f"{pct:+.2f}%"  # includes + or - sign
            color = "green" if pct > 0 else "red" if pct < 0 else "gray"
            ax.text(x, y, f"\n{pct_label}", fontsize=7, color=color,
                    ha='center', va='top', rotation=0)

    ax.set_title(f"{name} Price")
    ax.set_ylabel("Price (USD)")
    ax.legend()

plt.xticks(rotation=45)
plt.xlabel("Date")

plt.tight_layout()
plt.show()